```
File: linked_in_api.ipynb
Author: Michael Lucky
Date: September 13, 2023
Description: This notebook is intended to be used as a template for using the LinkedIn API to extract data from LinkedIn company profiles for data collection.

Copyright (c) 2023 Jelloow

For inquiries or permissions regarding the use of this code, please contact:
info@jelloow.com
```

In [1]:
import http.client
import logging
import tomllib

# format logging
logging.basicConfig(format='%(asctime)s: %(name)s: %(levelname)s: %(message)s', level=logging.DEBUG, datefmt='%Y-%m-%dT%H:%M:%S%z')

# create logger
log = logging.getLogger(__name__)

# read in .env file
with open('./.env', 'rb') as f:
    env = tomllib.load(f)

In [2]:
base_url = "api.linkedin.com"
version = "202206"
access_token = env["access_token"]

conn = http.client.HTTPSConnection(base_url)
payload = ''
headers = {
    'Authorization': f'Bearer {access_token}',
    'LinkedIn-Version': f'{version}',
    'X-Restli-Protocol-Version': '2.0.0'
}

domain = "gmail.com"
conn.request(
    "GET", f"/rest/organizations?q=emailDomain&emailDomain={domain}",
    payload,
    headers
)

res = conn.getresponse()
if res.status >= 200 and res.status < 300:
    data = res.read()
    log.debug(data.decode("utf-8"))
else:
    log.debug(f"Error: {res.status} {res.reason}")

2023-09-13T09:32:06-0600: __main__: DEBUG: Error: 401 Unauthorized
